## **손글씨 숫자 이미지 분류 - by Pytorch**
- https://tutorials.pytorch.kr/beginner/basics/quickstart_tutorial.html

## [ 단층신경망 - 선형회귀 사용]

`torch.nn` : **Neural Network(신경망)**를 구성하고 동작하게 하는 데 필요한 다양한 구성 요소와 기능들을 제공하는 **모듈**로 신경망을 구축하는 데 필요한 기본적인 **레이어, 활성화 함수, 손실 함수** 등을 포함하고 있습니다.

1. **레이어 (Layers)**:
   - `nn.Linear`: 선형 변환을 수행하는 완전 연결층 (Fully Connected Layer).
   - `nn.Conv2d`: 2D 합성곱(Convolution)을 수행하는 합성곱층.
   - `nn.RNN`, `nn.LSTM`, `nn.GRU`: 순환 신경망(RNN) 레이어들.

2. **활성화 함수 (Activation Functions)**:
   - `nn.ReLU`: ReLU (Rectified Linear Unit) 활성화 함수.
   - `nn.Sigmoid`: 시그모이드 활성화 함수.
   - `nn.Softmax`: 소프트맥스 활성화 함수.

3. **손실 함수 (Loss Functions)**:
   - `nn.CrossEntropyLoss`: 교차 엔트로피 손실 함수.
   - `nn.MSELoss`: 평균 제곱 오차 손실 함수.

4. **컨테이너 (Containers)**:
   - `nn.Sequential`: 여러 레이어를 순차적으로 쌓아 하나의 모델로 정의.
   - `nn.Module`: 모든 신경망 모듈의 기본 클래스. 사용자 정의 신경망을 만들 때 상속받아 사용.

## 1.라이브러리 가져오기

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import random

import warnings
warnings.filterwarnings("ignore")

## 2.데이터 로딩 및 전처리
- torchvision 라이브러리를 사용하여 MNIST 데이터셋을 다운로드
- train=True로 설정하면 훈련용 데이터셋(60,000개의 이미지)을 불러옵니다. False로 설정하면 테스트용 데이터셋(10,000개의 이미지)을 불러옵니다.
- transforms.ToTensor()는 이미지 데이터를 텐서로 변환합니다.
    - 이미지의 픽셀 값이 [0, 255] 범위에서 [0.0, 1.0] 범위로 정규화됨
    - MNIST 이미지는 원래 28x28 크기의 흑백 이미지로, 이를 [1, 28, 28] 크기의 3차원 텐서(채널, 높이, 너비)로 변환
- 배치 크기 단위로 데이터 로더를 사용해 데이터를 불러옵니다.

In [ ]:
# 배치 사이즈 설정
batch_size = 100
# 데이터 저장 경로 설정
root = './data'

# MNIST 학습/테스트 데이터셋 다운로드 및 로드, 텐서로 변환
mnist_train = dset.MNIST(root=root, train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dset.MNIST(root=root, train=False, transform=transforms.ToTensor(), download=True)

# 학습/테스 데이터 로더 설정 (데이터를 배치 크기만큼 나누고, 셔플하여 로드)
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, drop_last=True)

# GPU 사용 가능 시 CUDA를 사용하고, 그렇지 않으면 CPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device", device)

## 3.모델 정의하기

- 간단한 선형 모델(Linear)을 정의하여 28x28 이미지를 10개의 클래스(숫자 0~9)로 분류하도록 합니다.
- MNIST 입력의 크기는 28x28입니다.
- 여기서 구현하는 linear 모델은 입력이 1차원이기 때문에 입력 차원을 맞춰야합니다.

In [ ]:
# 단순 선형 모델 정의 (28*28 크기의 이미지를 10개의 클래스로 분류)
linear = torch.nn.Linear(28*28, 10, bias=True).to(device)
# 가중치를 정규분포로 초기화 (in-place)
torch.nn.init.normal(linear.weight)

In [ ]:
print(train_loader)

- 모델 학습 과정을 위한 옵티마이저와 로스 함수 지정하기
    - 옵티마이저는 SGD, Loss는 Cross Entropy Loss를 사용합니다.

In [ ]:
# 손실 함수로 교차 엔트로피 사용
criterion = torch.nn.CrossEntropyLoss().to(device)
# 옵티마이저로 SGD 사용, 학습률 0.1
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)

## 4.모델학습하기
- 구현 함수들을 이용해 학습 Loop를 구현해보세요.
    - 손실 함수로 CrossEntropyLoss를 사용하며, SGD 옵티마이저를 사용해 모델 파라미터를 업데이트합니다.

In [ ]:
# 학습 에포크 수 설정
training_epochs = 15

# 학습 과정의 손실과 정확도를 추적하기 위한 리스트 초기화
loss_history = []
accuracy_history = []

# 학습 루프 시작
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = len(train_loader)

    epoch_loss = 0
    epoch_accuracy = 0

    # 미니 배치 단위로 학습
    for i, (imgs, labels) in enumerate(train_loader):
        # 입력 이미지와 레이블을 GPU 또는 CPU로 이동
        imgs, labels = imgs.to(device), labels.to(device)
        # 이미지를 1차원 벡터로 변환 (28x28 -> 784)
        imgs = imgs.view(-1, 28 * 28)

        # 모델을 사용하여 예측값 계산
        outputs = linear(imgs)
        # 손실 계산
        loss = criterion(outputs, labels)

        # 옵티마이저의 그래디언트 초기화
        optimizer.zero_grad()
        # 손실을 역전파하여 그래디언트 계산
        loss.backward()
        # 옵티마이저로 가중치 업데이트
        optimizer.step()

        # 배치 손실 및 정확도 계산
        epoch_loss += loss.item()
        # 모델 예측 -  outputs : [0.1, 0.2, 0.05, 0.9, 0.4, 0.3, 0.2, 0.1, 0.1, 0.1]이라면, torch.max(outputs, 1)은 (0.9, 3)을 반환
        _, argmax = torch.max(outputs, 1)
        accuracy = (labels == argmax).float().mean()   # 부동 소수점 텐서로 변환 : True는 1.0, False는 0.0
        epoch_accuracy += accuracy.item()

        # 100번의 반복마다 손실과 정확도 출력
        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step[{}/{}, Loss: {:.4f}, Accuracy: {:.2f}%'.format(
                epoch + 1, training_epochs, i + 1, len(train_loader), loss.item(), accuracy.item() * 100
            ))

    # 에포크마다 평균 손실과 정확도 기록
    epoch_loss /= total_batch
    epoch_accuracy /= total_batch
    loss_history.append(epoch_loss)
    accuracy_history.append(epoch_accuracy)

## 5.학습 과정 시각화

In [ ]:
# 학습 과정의 손실과 정확도를 그래프로 시각화
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(range(training_epochs), loss_history, label='Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')

plt.subplot(1, 2, 2)
plt.plot(range(training_epochs), accuracy_history, label='Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')

plt.show()

## 6.모델 평가하기
- 학습이 끝난 후 테스트 데이터셋을 사용해 모델의 성능을 평가하고, 임의로 선택한 이미지에 대한 예측 결과를 출력합니다.

In [ ]:
# 모델을 평가 모드로 전환
linear.eval()

# 평가 시 그래디언트 계산하지 않음
with torch.no_grad():             # torch.no_grad() : gradient를 계산안함
    correct = 0    # 모델이 올바르게 예측한 이미지의 수를 누적하기 위한 변수
    total = 0      # 테스트한 전체 이미지의 수를 누적하기 위한 변수

    # 테스트 데이터셋을 사용하여 정확도 평가
    for i, (t_imgs, t_labels) in enumerate(test_loader):
        t_imgs, t_labels = t_imgs.to(device), t_labels.to(device)
        # 28x28 크기의 2차원 이미지를 (batch_size, 784) 크기의 1차원 벡터로 변환
        t_imgs = t_imgs.view(-1, 28 * 28)

        # 테스트 이미지에 대한 예측값 계산
        prediction = linear(t_imgs)

        # 예측된 레이블과 실제 레이블 비교하여 정확도 계산
        _, argmax = torch.max(prediction, 1)
        total += t_imgs.size(0)    # t_imgs.size(0) : 배치 사이즈
        correct += (t_labels == argmax).sum().item()  # .sum():맞은 예측의 수 계산,.item()은 그 값을 파이썬의 숫자로 변환 - tensor(5.0)인 경우, 5.0이라는 숫자를 추출하여 파이썬의 float 타입으로 변환

    # 전체 테스트 데이터에 대한 정확도 출력
    print("Test accuracy for {} images: {:.2f}%".format(total, correct / total*100))

## 7.모델 예측하기

In [ ]:
# 테스트 데이터의 임의 샘플에 대해 예측 결과 시각화
r = random.randint(0, len(mnist_test) - 1)  # r:테스트 데이터에서 무작위로 선택한 이미지의 인덱스번호

t_imgs_data = mnist_test.test_data[r: r + 1].view(-1, 28 * 28).float().to(device)
t_labels_data = mnist_test.test_labels[r:r + 1].to(device)

print("Label: ", t_labels_data.item())  # 이블 값을 텐서에서 추출하여 파이썬의 정수로 변환

# 모델 예측 수행
single_prediction = linear(t_imgs_data)
print("Prediction: ", torch.argmax(single_prediction, 1).item())

plt.imshow(mnist_test.test_data[r:r + 1].view(28, 28), cmap="Greys", interpolation="nearest")   # interpolation="nearest" : 이미지를 확대할 때 픽셀의 경계가 뚜렷하게 유지되도록 하는 옵션
plt.show()

## [ 심층신경망, DNN ]

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets as dset
from torchvision import transforms
from torch.utils.data import DataLoader

# 배치 사이즈 설정
batch_size = 100

# 데이터 저장 경로 설정
root = './data'

# MNIST 학습/테스트 데이터셋 다운로드 및 로드, 텐서로 변환
mnist_train = dset.MNIST(root=root, train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dset.MNIST(root=root, train=False, transform=transforms.ToTensor(), download=True)

# 학습/테스트 데이터 로더 설정 (데이터를 배치 크기만큼 나누고, 셔플하여 로드)
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=True, drop_last=True)

# GPU 사용 가능 시 CUDA를 사용하고, 그렇지 않으면 CPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:", device)

# 모델 정의 (히든 레이어 추가)
class SimpleDNN(nn.Module):
    def __init__(self):
        super(SimpleDNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)  # 첫 번째 히든 레이어 (128 유닛)
        self.relu = nn.ReLU()             # ReLU 활성화 함수
        self.fc2 = nn.Linear(128, 10)     # 출력 레이어 (10 유닛, 클래스 수)

    def forward(self, x):
        x = self.fc1(x)   # 입력을 첫 번째 히든 레이어에 전달
        x = self.relu(x)  # ReLU 활성화 함수 적용
        x = self.fc2(x)   # 히든 레이어 출력을 출력 레이어에 전달
        return x

# 모델 초기화
model = SimpleDNN().to(device)

# 손실 함수, 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 학습 에포크 수 설정
training_epochs = 15

# 학습 과정의 손실과 정확도를 추적하기 위한 리스트 초기화
loss_history = []
accuracy_history = []

# 학습 루프 시작
for epoch in range(training_epochs):
    epoch_loss = 0
    epoch_accuracy = 0

    # 미니 배치 단위로 학습
    for i, (imgs, labels) in enumerate(train_loader):
        # 입력 이미지와 레이블을 GPU 또는 CPU로 이동
        imgs, labels = imgs.to(device), labels.to(device)
        # 이미지를 1차원 벡터로 변환 (28x28 -> 784)
        imgs = imgs.view(-1, 28 * 28)

        # 모델을 사용하여 예측값 계산
        outputs = model(imgs)
        # 손실 계산
        loss = criterion(outputs, labels)

        # 옵티마이저의 그래디언트 초기화
        optimizer.zero_grad()
        # 손실을 역전파하여 그래디언트 계산
        loss.backward()
        # 옵티마이저로 가중치 업데이트
        optimizer.step()

        # 배치 손실 및 정확도 계산
        epoch_loss += loss.item()
        _, argmax = torch.max(outputs, 1)
        accuracy = (labels == argmax).float().mean()
        epoch_accuracy += accuracy.item()

        # 100번의 반복마다 손실과 정확도 출력
        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{training_epochs}], '
                  f'Step[{i + 1}/{len(train_loader)}], '
                  f'Loss: {loss.item():.4f}, Accuracy: {accuracy.item() * 100:.2f}%')

    # 에포크별 평균 손실 및 정확도 기록
    loss_history.append(epoch_loss / len(train_loader))
    accuracy_history.append(epoch_accuracy / len(train_loader))

# 학습이 끝나면 테스트 데이터로 평가
model.eval()
with torch.no_grad():
    test_loss = 0
    test_accuracy = 0
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28 * 28)
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, argmax = torch.max(outputs, 1)
        accuracy = (labels == argmax).float().mean()
        test_accuracy += accuracy.item()
    test_loss /= len(test_loader)
    test_accuracy /= len(test_loader)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy * 100:.2f}%")
